In [3]:
!pip install pandas numpy requests faiss-cpu -q

In [6]:
import faiss
import requests
import numpy as np
import pandas as pd

In [9]:
print(f'Pandas: {pd.__version__}\nNumpy: {np.__version__}\nRequests: {requests.__version__}\nFaiss: {faiss.__version__}')

Pandas: 2.2.2
Numpy: 1.26.3
Requests: 2.32.3
Faiss: 1.8.0

In [4]:
import pandas as pd

df = pd.read_csv('books.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6810 entries, 0 to 6809
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isbn13          6810 non-null   int64  
 1   isbn10          6810 non-null   object 
 2   title           6810 non-null   object 
 3   subtitle        2381 non-null   object 
 4   authors         6738 non-null   object 
 5   categories      6711 non-null   object 
 6   thumbnail       6481 non-null   object 
 7   description     6548 non-null   object 
 8   published_year  6804 non-null   float64
 9   average_rating  6767 non-null   float64
 10  num_pages       6767 non-null   float64
 11  ratings_count   6767 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 638.6+ KB


In [8]:
df.thumbnail[2]

'http://books.google.com/books/content?id=OmQawwEACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api'

In [5]:
df.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [11]:
def text_rep(row):  #textual representatins of all required columns values in a row
    text_rep = f'''Tile: {row['title']}
Authors: {row['authors']}
Description: {row['description']}
Categories: {row['categories']}
Publishing Year: {row['published_year']}
Average Rating: {row['average_rating']}
Number of Pages: {row['num_pages']}
'''
    return text_rep

In [17]:
from rich import print
print(df.iloc[:3].apply(text_rep, axis=1).values[1])

Tile: Spider's Web
Authors: Charles Osborne;Agatha Christie
Description: A new 'Christie for Christmas' -- a full-length novel adapted from her acclaimed play by Charles 
Osborne Following BLACK COFFEE and THE UNEXPECTED GUEST comes the final Agatha Christie play novelisation, bringing
her superb storytelling to a new legion of fans. Clarissa, the wife of a Foreign Office diplomat, is given to 
daydreaming. 'Supposing I were to come down one morning and find a dead body in the library, what should I do?' she
muses. Clarissa has her chance to find out when she discovers a body in the drawing-room of her house in Kent. 
Desperate to dispose of the body before her husband comes home with an important foreign politician, Clarissa 
persuades her three house guests to become accessories and accomplices. It seems that the murdered man was not 
unknown to certain members of the house party (but which ones?), and the search begins for the murderer and the 
motive, while at the same time trying to persuade a police inspector that there has been no murder at all... 
SPIDER'S WEB was written in 1954 specifically for Margaret Lockwood and opened first at the Theatre Royal 
Nottingham before moving to the Savoy Theatre in London on 14 December 1954. With THE MOUSETRAP and WI
Categories: Detective and mystery stories
Publishing Year: 2000.0
Average Rating: 3.83
Number of Pages: 241.0

In [18]:
## Apply above function on whole dataset
df['textual_representation'] = df.apply(text_rep, axis=1)

In [19]:
df['textual_representation'].head()

0    Tile: Gilead\nAuthors: Marilynne Robinson\nDes...
1    Tile: Spider's Web\nAuthors: Charles Osborne;A...
2    Tile: The One Tree\nAuthors: Stephen R. Donald...
3    Tile: Rage of angels\nAuthors: Sidney Sheldon\...
4    Tile: The Four Loves\nAuthors: Clive Staples L...
Name: textual_representation, dtype: object

In [23]:
!ollama list

NAME                                 	ID          	SIZE  	MODIFIED      
mxbai-embed-large:latest             	468836162de7	669 MB	5 minutes ago	
llava:13b                            	0d0eb4d7f485	8.0 GB	13 days ago  	
bge-m3:latest                        	790764642607	1.2 GB	2 weeks ago  	
qwen2:7b-instruct-fp16               	99954bc7f7d6	15 GB 	3 weeks ago  	
qwen2:7b-instruct-q8_0               	2f9a76350a1f	8.1 GB	3 weeks ago  	
qwen2:7b-instruct-q6_K               	fb2f17833020	6.3 GB	3 weeks ago  	
qwen2:latest                         	e0d4e1163c58	4.4 GB	3 weeks ago  	
mistral-large:123b-instruct-2407-q6_K	5e4615a812a8	100 GB	3 weeks ago  	
mistral-large:latest                 	0ca7dfa0bf06	69 GB 	3 weeks ago  	
gemma2:latest                        	ff02c3702f32	5.4 GB	3 weeks ago  	
gemma2:2b                            	8ccf136fdd52	1.6 GB	3 weeks ago  	
llama3:latest                        	365c0bd3c000	4.7 GB	3 weeks ago  	
mistral-nemo:latest                  	994f3b8b7801	

In [25]:
import faiss
import requests
import numpy as np
from tqdm import tqdm

dim = 1024

index = faiss.IndexFlatL2(dim)

X = np.zeros((len(df['textual_representation']), dim), dtype='float32')

In [26]:
for i, representation in tqdm(enumerate(df['textual_representation']), total=len(df['textual_representation'])):
    res = requests.post('http://localhost:11434/api/embeddings',
                        json={
                            'model': 'mxbai-embed-large:latest',
                            'prompt': representation
                        })
    embedding = res.json()['embedding']

    X[i] = np.array(embedding)

index.add(X)

100%|██████████| 6810/6810 [11:50<00:00,  9.59it/s]


In [27]:
faiss.write_index(index, 'index')

In [28]:
index = faiss.read_index('index')

In [30]:
df[df.title.str.contains('Friends')]

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,textual_representation
502,9780064420808,0064420809,Little House Friends,NaN,Heather Henson;Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=gMZbA...,Laura Ingalls shares adventures and good times...,1998.0,3.99,80.0,70.0,Tile: Little House Friends\nAuthors: Heather H...
612,9780099498599,0099498596,Circle of Friends,NaN,Maeve Binchy,College attendance,http://books.google.com/books/content?id=P4-i_...,"It began with Benny Hogan and Eve Malone, grow...",2006.0,4.02,722.0,50668.0,Tile: Circle of Friends\nAuthors: Maeve Binchy...
986,9780142300848,0142300845,"Oliver and Albert, Friends Forever",NaN,Jean Van Leeuwen,Juvenile Fiction,http://books.google.com/books/content?id=-EBFs...,"Oliver makes friends with Albert, the new boy ...",2002.0,3.62,48.0,22.0,"Tile: Oliver and Albert, Friends Forever\nAuth..."
1776,9780345323903,0345323904,With Friends Like These,NaN,Alan Dean Foster,Fiction,http://books.google.com/books/content?id=IKUcA...,Willie Whitehorse could have been just another...,1984.0,3.94,236.0,2233.0,Tile: With Friends Like These\nAuthors: Alan D...
2490,9780394895833,0394895835,Baby's Animal Friends,NaN,Phoebe Dunn,Juvenile Fiction,http://books.google.com/books/content?id=ytIAZ...,Photographs capture the special relationship b...,1988.0,3.40,28.0,15.0,Tile: Baby's Animal Friends\nAuthors: Phoebe D...
4373,9780743272773,0743272773,How to Win Friends and Influence People for Te...,NaN,Donna Dale Carnegie,Self-Help,http://books.google.com/books/content?id=eIc2l...,"Donna Dale Carnegie, daughter of the late moti...",2005.0,3.90,208.0,426.0,Tile: How to Win Friends and Influence People ...
4533,9780749307844,0749307846,How to Win Friends and Influence People,NaN,Dale Carnegie,Conduct of life,http://books.google.com/books/content?id=aO7CQ...,Dale Carnegie aims to show how to makes friend...,1990.0,4.18,256.0,199.0,Tile: How to Win Friends and Influence People\...
5115,9780810958623,0810958627,Ruby Gloom's Guide to Friendship,NaN,Mighty Fine Inc.,Juvenile Nonfiction,http://books.google.com/books/content?id=FYrcu...,"If Ruby Gloom's friends seem somewhat unusual,...",2005.0,4.34,72.0,37.0,Tile: Ruby Gloom's Guide to Friendship\nAuthor...


In [31]:
# fav book
favorite_book = df.iloc[1357]

In [34]:
print(favorite_book.textual_representation)  

Tile: Elbow Room
Authors: Daniel Clement Dennett
Description: Essays discuss reason, self-control, self-definition, time, cause and effect, accidents, and 
responsibility, and explain why people want free will
Categories: Philosophy
Publishing Year: 1984.0
Average Rating: 3.92
Number of Pages: 200.0

In [35]:
res = requests.post('http://localhost:11434/api/embeddings',
                        json={
                            'model': 'mxbai-embed-large:latest',
                            'prompt': favorite_book['textual_representation']
                        })

In [36]:
embedding = np.array([res.json()['embedding']], dtype='float32')

In [38]:
D, I = index.search(embedding, 7)

In [39]:
best_matches = np.array(df['textual_representation'])[I.flatten()]

In [41]:
for match in best_matches:
    print(match)
    print()

Tile: Elbow Room
Authors: Daniel Clement Dennett
Description: Essays discuss reason, self-control, self-definition, time, cause and effect, accidents, and 
responsibility, and explain why people want free will
Categories: Philosophy
Publishing Year: 1984.0
Average Rating: 3.92
Number of Pages: 200.0

Tile: Huxley and God
Authors: Aldous Huxley
Description: This volume of essays, written with the authors trademark elegance and wit, tackles subjects such as 
Action and Contemplation, Religion and Time, Reflections on the Lord's Prayer, and Notes on Zen.
Categories: Religion
Publishing Year: 2003.0
Average Rating: 4.31
Number of Pages: 320.0

Tile: The Mind's I
Authors: Douglas R. Hofstadter;Daniel Clement Dennett
Description: Brilliant, shattering, mind-jolting, "The Mind's I" is a searching, probing nook--a cosmic journey of 
the mind--that goes deeply into the problem of self and self-consciousness as anything written in our time. From 
verbalizing chimpanzees to scientific speculations involving machines with souls, from the mesmerizing, maze-like 
fiction of Borges to the tantalizing, dreamlike fiction of Lem and Princess Ineffable, her circuits glowing read 
and gold, "The Mind's I " opens the mind to the Black Box of fantasy, to the windfalls of reflection, to new 
dimensions of exciting possibilities.
Categories: Philosophy
Publishing Year: 1982.0
Average Rating: 4.14
Number of Pages: 512.0

Tile: Philosophical Papers: Volume 1, Human Agency and Language
Authors: Charles Taylor
Description: Philosophical Papers will interest a very wide range of philosophers and students of the human 
sciences.
Categories: Philosophy
Publishing Year: 1985.0
Average Rating: 4.31
Number of Pages: 304.0

Tile: Beyond the Postmodern Mind
Authors: Huston Smith
Description: This new edition of critically acclaimed essays explores possible breakthroughs in the direction of 
reaching a liberated and enlightened consciousness. With a new preface and new final chapter. An invaluable 
collection of essays by the foremost religious writer in America today.
Categories: Philosophy
Publishing Year: 2003.0
Average Rating: 4.21
Number of Pages: 295.0

Tile: For the Sake of Argument
Authors: Christopher Hitchens
Description: Essays discuss international politics, modern conservatism, politicians, and literature
Categories: Literary Collections
Publishing Year: 1994.0
Average Rating: 4.17
Number of Pages: 368.0

Tile: The World as Will and Representation
Authors: Arthur Schopenhauer
Description: The German philosopher explains his thoughts about intellectual perception and abstract representation
and critically analyzes Kant's ideas and teachings. Bibliogs
Categories: Philosophy
Publishing Year: 1958.0
Average Rating: 4.18
Number of Pages: 534.0

## **Extra**

In [43]:
from tqdm import tqdm
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to get embeddings asynchronously
def get_embedding(representation, i):
    res = requests.post('http://localhost:11434/api/embeddings',
                        json={
                            'model': 'mxbai-embed-large:latest',
                            'prompt': representation
                        })
    embedding = res.json()['embedding']
    return i, np.array(embedding)

import faiss
import requests
import numpy as np
from tqdm import tqdm

dim = 1024

X = np.zeros((len(df['textual_representation']), dim), dtype='float32')

# Set up ThreadPoolExecutor for parallel requests
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit all tasks to the executor
    futures = [executor.submit(get_embedding, representation, i) for i, representation in enumerate(df['textual_representation'])]

    # Use tqdm to display progress as tasks complete
    for future in tqdm(as_completed(futures), total=len(futures)):
        i, embedding = future.result()  # Get result from future
        X[i] = embedding

# Assuming `index` is your vector index
index.add(X)


 17%|█▋        | 1163/6810 [00:38<03:07, 30.04it/s]
